# Ejemplo(Falta agregar un costo de almacenamiento)
Juan vende cervezas artesanales de Lunes a Viernes. Las compra por 300 pesos y vende por 1.500 pesos cada una.
Juan maneja su inventario de forma semanal, de la siguiente manera:
* El Lunes al comienzo del día Juan hace la contabilidad de costos, y cuenta cuantas unidades tiene en inventario. Según su nivel de inventario, tiene que tomar una decisión.
    
    * Comprar una cantidad i para atender la clientela la semana j, sin sobrepasar su capacidad de S unidades.
    * No hacer ningún pedido esta semana.
* Finalmente el pedido lo hace el lunes en la mañana y llega en ese mismo instante, Juan vende las cervezas durante la semana.

    
* Si un cliente quiere comprar una cerveza pero no hay disponible, Juan consideraque pierde 1200 pesos por cada cerveza no vendida por falta de inventario. (Precio de Venta - Costo de compra)
    
* Juan vende entre 0 y 400 cervezas por semana. (De momento demanda Uniforme)
    

* El frigobar de juan tiene una capacidad de 500 cervezas (S=500)
* Si la demanda durante la semana sobrepasa el inventario se considera una perdida del beneficio que podría haber recibido ((Precio venta - Precio Compra) * Cantidad de cervezas fuera de inventario)
* Juan pide dinero prestado, por lo cual considera una tasa de descuento del dinero de un 10% semanal

## ¿Ayude a decidir a Juan cuando comprar, según el inventario que tenga?

In [7]:
import numpy as np
from numpy import random
import collections
import time

In [8]:
# Información problema
S=500#00#20#0 #capacidad max y tamaño de lote
a=0#00#0#3#0 #Demanda minima
b=400#00#15#0 #Demanda maxima
cost=300#precio cerveza y costo perdida clientes
price=1500
taza=0.9


#### Generó las demandas

In [9]:
random.seed(10)
total=50000
x=random.randint(a,b+1, size=(total))
Dem_frec = collections.Counter(x)

### Comiezo a generar mi matriz de probaibilidades
Esta matriz se estructura de la siguiente forma

    Prob[i][j]: es la probailidad de que el inventario i pase al nivel de inventario j en el siguente periodo


In [10]:
prob =np.zeros([S+1,S+1])
for i in range(S+1):
    for j in range(S+1):
        dem=i-j
        if(S>=dem>=0):
            if(dem<i):
               prob[i][j] =Dem_frec[dem]/total
    prob[i][0] = 1 - sum(prob[i][0:])




Tomando en cuenta que realizar la compra en el periodo t, donde el nivel de inventarios es de i, no afectaria al expected reward del mismo periodo t, ya que la compra afecta al nivel de inventarios del perido siguente. Es por esto que la desición no afecta al expected reward del mismo periodo en donde se toma.

Es por ello que para almacenar el expected reward, se utilizará al siguiente estructura:
    
    reward[i]:= el expected reward ante un nivel de inventario i.


In [11]:
# aquí se ajustan los costos de
# D>i o D<i

reward=np.zeros(S+1)
for i in range(S+1):
    r=0
    for dem in range(a,b+1):
        p=Dem_frec[dem]/total
        r_dem=price*min(dem,i)
        if(dem>=i):
            r_dem+=-(p-cost)*(dem-i)
        else:
            #if(dem<i):#Agrego costo de inventario sobrante
            r_dem+= -(cost)*(i-dem)
        r+= p*(r_dem)
    #print(r)
    reward[i]=r

# Comenzamos a estructurar esto

## Value Determination Equations

In [12]:
#funcion para obtener el valor de la ecuaciones
def Value_Determination_Equations(decisions,prob,descuento,costo,revenues,S):
    L=len(decisions)
    aux=np.zeros([L,L])
    B=np.zeros(L)
    for i in range(L):
        cant_comprada_i=decisions[i]
        if cant_comprada_i>0:
        #Juan decide comprar mas cerveza.
            if(cant_comprada_i+i<=S):
            # La compra más el inventario debe ser menor a Cap max
                B[i]=revenues[cant_comprada_i+i]
                B[i]+=-costo*(cant_comprada_i)
                aux[i]=prob[cant_comprada_i+i]
            else:
                B[i]+=-costo*(cant_comprada_i)*10000
                aux[i]=prob[i]
        else:
            aux[i]=prob[i]
            B[i]=revenues[i]
    A = np.identity(L)-(descuento*aux)
    X = np.linalg.solve(A,B)
    return X
    

# Howard’s Policy Iteration Method

In [13]:
def T_delta(prob,values,descuento,costo,revenues):
    L=len(values)
    T=np.zeros(L)
    Cant=np.zeros(L)
    for i in range(L):
        lista=[]
        lista_2=[]
        for j in range(i,L):
            lista.append(revenues[j]-costo*(j-i)+(descuento*sum(prob[j]*values)))
            lista_2.append(j-i)
        T[i]=max(lista)#Con i=S soy indiferente a comprar o no
        indice=lista.index(T[i])
        Cant[i]=lista_2[indice]
    return [T,Cant]

In [14]:
def Search_OP(decisions,prob,descuento,costo,revenues,S):
   
    #(D,m_prob,descuento,c,states_reveneus)
    V_D_E = Value_Determination_Equations(decisions,prob,descuento,costo,revenues,S)
    Los_T, D_T = T_delta(prob,V_D_E,descuento,costo,revenues)
    val_Op=np.around(Los_T,4)==np.around(V_D_E,4)
    D_p = np.copy(decisions)
    if(sum(val_Op)==len(val_Op)):
        return(decisions)
    else:
        for i in range(len(val_Op)):
            if (val_Op[i]!=True):
                decisions[i]=D_T[i].astype('int')
        return(Search_OP(decisions,prob,descuento,costo,revenues,S))
        #For 
        #print('recurción')        
        

In [15]:
D=np.ones(S+1).astype('int')
start_time = time.time()
Optimo=Search_OP(D,prob,taza,cost,reward,S)
print("--- %s Segundos ---" % (time.time() - start_time))
Optimo

--- 51.40433716773987 Segundos ---


array([291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 281, 280, 279,
       278, 277, 276, 275, 274, 273, 272, 271, 270, 269, 268, 267, 266,
       265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253,
       252, 251, 250, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240,
       239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227,
       226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214,
       213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201,
       200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188,
       187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175,
       174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162,
       161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149,
       148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136,
       135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123,
       122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 11

--- 47.435879945755005 seconds ---
array([291, 290, 289, 288, 287, 286, 285, 284, 283, 282, 281, 280, 279,
       278, 277, 276, 275, 274, 273, 272, 271, 270, 269, 268, 267, 266,
       265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253,
       252, 251, 250, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240,
       239, 238, 237, 236, 235, 234, 233, 232, 231, 230, 229, 228, 227,
       226, 225, 224, 223, 222, 221, 220, 219, 218, 217, 216, 215, 214,
       213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201,
       200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188,
       187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175,
       174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162,
       161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149,
       148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136,
       135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123,
       122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110,
       109, 108, 107, 106, 105, 104, 103, 102, 101, 100,  99,  98,  97,
        96,  95,  94,  93,  92,  91,  90,  89,  88,  87,  86,  85,  84,
        83,  82,  81,  80,  79,  78,  77,  76,  75,  74,  73,  72,  71,
        70,  69,  68,  67,  66,  65,  64,  63,  62,  61,  60,  59,  58,
        57,  56,  55,  54,  53,  52,  51,  50,  49,  48,  47,  46,  45,
        44,  43,  42,  41,  40,  39,  38,  37,  36,  35,  34,  33,  32,
        31,  30,  29,  28,  27,  26,  25,  24,  23,  22,  21,  20,  19,
        18,  17,  16,  15,  14,  13,  12,  11,  10,   9,   8,   7,   6,
         5,   4,   3,   2,   1,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])